In [2]:
import numpy as np
import pandas as pd
import cv2

In [3]:
from pytube import YouTube

In [4]:
#YouTube('https://www.youtube.com/watch?v=3iiodzoG80A').streams.first().download()

In [15]:
cap = cv2.VideoCapture('OpenCV People Counting Demo 2.webm')

while(cap.isOpened()):
    ret, frame = cap.read()
    
    try:
        cv2.imshow('frame',frame)
    except:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
print('done')

done


In [6]:
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
from darknet import Darknet
from preprocess import prep_image, inp_to_image, letterbox_image
import pandas as pd
import random 
import pickle as pkl
import argparse

In [7]:
def get_test_input(input_dim, CUDA):
    img = cv2.imread("dog-cycle-car.png")
    img = cv2.resize(img, (input_dim, input_dim)) 
    img_ =  img[:,:,::-1].transpose((2,0,1))
    img_ = img_[np.newaxis,:,:,:]/255.0
    img_ = torch.from_numpy(img_).float()
    img_ = Variable(img_)    
    return img_

In [8]:
def prep_image(img, inp_dim) -> 'valiable':
    """
    Prepare image for inputting to the neural network. 
    """
    orig_im = img
    dim = orig_im.shape[1], orig_im.shape[0]
    img = (letterbox_image(orig_im, (inp_dim, inp_dim)))
    img_ = img[:,:,::-1].transpose((2,0,1)).copy()
    img_ = torch.from_numpy(img_).float().div(255.0).unsqueeze(0)
    return img_, orig_im, dim

In [9]:
def write(x, img):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    cls = int(x[-1])
    label = "{0}".format(classes[cls])
    color = random.choice(colors)
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img

In [82]:
confidence = 0.6
nms_thesh = 0.2
start = 0
CUDA = torch.cuda.is_available()
num_classes = 1
bbox_attrs = 5 + num_classes

cfgfile = 'cfg/yolov3.cfg'
weightsfile = 'yolov3.weights'

reso = '96'

In [83]:
print("Loading network.....")
model = Darknet(cfgfile)
model.load_weights(weightsfile)
print("Network successfully loaded")

Loading network.....
Network successfully loaded


In [84]:
model.net_info["height"] = reso
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0 
assert inp_dim > 32

In [85]:
#model(get_test_input(inp_dim, CUDA), CUDA)
#model.eval()

In [86]:
classes = load_classes('data/coco.names')
colors = pkl.load(open("pallete", "rb"))

In [87]:
cap.release()
cv2.destroyAllWindows()

In [88]:
#videofile = 'video.avi'
cap = cv2.VideoCapture('OpenCV People Counting Demo 2.webm')
assert cap.isOpened()

frames = 0
start = time.time()    
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        img, orig_im, dim = prep_image(frame, inp_dim)
        im_dim = torch.FloatTensor(dim).repeat(1, 2)                        

        output = model(Variable(img, volatile = True), CUDA)
        output = write_results(output, 
                               confidence, 
                               num_classes, 
                               nms = True, 
                               nms_conf = nms_thesh)


        if type(output) == int:
            frames += 1
            print("FPS of the video is {:5.2f}".format( 
                frames / (time.time() - start)))
            cv2.imshow("frame", orig_im)
            key = cv2.waitKey(1)
            if key & 0xFF == ord('q'):
                break
            continue


        im_dim = im_dim.repeat(output.size(0), 1)
        scaling_factor = torch.min(inp_dim/im_dim,1)[0].view(-1,1)

        output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim[:,0].view(-1,1))/2
        output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim[:,1].view(-1,1))/2

        output[:,1:5] /= scaling_factor

        for i in range(output.shape[0]):
            output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim[i,0])
            output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim[i,1])




        list(map(lambda x: write(x, orig_im), output))


        cv2.imshow("frame", orig_im)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break
        frames += 1
        print("FPS of the video is {:5.2f}".format( frames / (time.time() - start)))


    else:
        break

/home/best/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  del sys.path[0]
/home/best/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


FPS of the video is  1.97
FPS of the video is  2.02
FPS of the video is  2.31
FPS of the video is  2.37
FPS of the video is  2.37
FPS of the video is  2.20
FPS of the video is  2.21
FPS of the video is  2.24
FPS of the video is  2.32
FPS of the video is  2.37
FPS of the video is  2.35
FPS of the video is  2.37
FPS of the video is  2.36
FPS of the video is  2.40
FPS of the video is  2.42
FPS of the video is  2.42
FPS of the video is  2.44
FPS of the video is  2.47
FPS of the video is  2.49
FPS of the video is  2.51
FPS of the video is  2.51
FPS of the video is  2.51
FPS of the video is  2.49
FPS of the video is  2.51
FPS of the video is  2.52
FPS of the video is  2.52
FPS of the video is  2.54
FPS of the video is  2.55
FPS of the video is  2.56
FPS of the video is  2.56
FPS of the video is  2.57
FPS of the video is  2.57
FPS of the video is  2.54
FPS of the video is  2.54
FPS of the video is  2.55
FPS of the video is  2.56
FPS of the video is  2.58
FPS of the video is  2.58
FPS of the v

KeyboardInterrupt: 